In [75]:
import numpy as np

In [76]:
import numpy as np

In [77]:
data=pd.read_csv("C://Users/Chandragupta//Desktop/dataset/train.csv")

In [78]:
data.head()

,News_ID,Date,Headline,Article,Journalists,market_change%
0,N_18841,2018-02-17 03:50:00,China insurer PICC debut shines through Hong K...,"' Shares in Chinese state', ""owned insurer PIC...",['Denny Thomas' 'Elzio Barreto'],-6.668293
1,N_17288,2018-01-12 02:49:00,AngloGold suspends operations at South Africa ...,""" AngloGold Ashanti ( ANGJ.J ), the world's th...",[''],0.042708
2,N_14657,2017-11-29 18:10:00,Boeing defense CEO says BAE-EADS merger needs ...,' A merger of Europe\'s EADS ( EAD.PA ) and Br...,['Andrea Shalal-Esa'],0.509376
3,N_17788,2018-01-24 17:12:00,Analysis: U.S. credit rating could again take ...,"' In 2011, the United States emerged from a da...",['Luciana Lopez'],2.512752
4,N_10558,2017-09-03 12:08:00,Volkswagen's finance arm ponders Greek euro-zo...,' Volkswagen\'s ( VOWG_p.DE ) finance division...,['Andreas Cremer'],0.262096


In [79]:
data.dtypes

News_ID            object
Date               object
Headline           object
Article            object
Journalists        object
market_change%    float64
dtype: object

In [80]:
data.isnull().any()

News_ID           False
Date              False
Headline          False
Article           False
Journalists       False
market_change%    False
dtype: bool

In [81]:
data.nunique()

News_ID           6000
Date              5867
Headline          6000
Article           6000
Journalists       1568
market_change%    6000
dtype: int64

In [82]:
data.describe()

,market_change%
count,6000.000000
mean,-0.397767
std,3.088720
min,-9.543868
25%,-2.768609
50%,-0.041615
75%,1.574347
max,5.918452


In [83]:
data["Date"]=data["Date"]+" "

In [84]:
data["joined"]=data["Date"]+data["Headline"].astype(str)+data["Article"].astype(str)

In [85]:
data.head()

,News_ID,Date,Headline,Article,Journalists,market_change%,joined
0,N_18841,2018-02-17 03:50:00,China insurer PICC debut shines through Hong K...,"' Shares in Chinese state', ""owned insurer PIC...",['Denny Thomas' 'Elzio Barreto'],-6.668293,2018-02-17 03:50:00 China insurer PICC debut s...
1,N_17288,2018-01-12 02:49:00,AngloGold suspends operations at South Africa ...,""" AngloGold Ashanti ( ANGJ.J ), the world's th...",[''],0.042708,2018-01-12 02:49:00 AngloGold suspends operati...
2,N_14657,2017-11-29 18:10:00,Boeing defense CEO says BAE-EADS merger needs ...,' A merger of Europe\'s EADS ( EAD.PA ) and Br...,['Andrea Shalal-Esa'],0.509376,2017-11-29 18:10:00 Boeing defense CEO says BA...
3,N_17788,2018-01-24 17:12:00,Analysis: U.S. credit rating could again take ...,"' In 2011, the United States emerged from a da...",['Luciana Lopez'],2.512752,2018-01-24 17:12:00 Analysis: U.S. credit rati...
4,N_10558,2017-09-03 12:08:00,Volkswagen's finance arm ponders Greek euro-zo...,' Volkswagen\'s ( VOWG_p.DE ) finance division...,['Andreas Cremer'],0.262096,2017-09-03 12:08:00 Volkswagen's finance arm p...


In [128]:
len(data["joined"][0])

5706

In [87]:
joine_data=data["joined"]

In [88]:
label=data["market_change%"]


In [89]:
df=data["joined"]

In [90]:
X=joine_data

In [91]:
y=label

In [92]:
import tensorflow as tf
from tensorflow import keras

In [93]:
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(output_sequence_length=1000)

In [94]:
d=tf.data.Dataset.from_tensor_slices(joine_data)
vectorize_layer.adapt(d.batch(64))

In [95]:
len(vectorize_layer.get_vocabulary())

61541

In [96]:
class BagOfWords(keras.layers.Layer):
    def __init__(self, n_tokens, dtype=tf.int32, **kwargs):
        super().__init__(dtype=tf.int32, **kwargs)
        self.n_tokens = n_tokens
    def call(self, inputs):
        one_hot = tf.one_hot(inputs, self.n_tokens)
        return tf.reduce_sum(one_hot, axis=1)[:, 1:]

In [97]:
bagofwords=BagOfWords(n_tokens=1002)

In [98]:
from tensorflow.keras.layers import Dense

In [99]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(bagofwords)
model.add(Dense(512, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(1))

In [100]:
model.compile(loss="mse", optimizer="nadam",
              metrics=["accuracy"])

In [101]:
history=model.fit(X[:2500],y[:2500],epochs=20,validation_data=(X[2500:4000],y[2500:4000]))

Train on 2500 samples, validate on 1500 samples
Epoch 1/20
2500/2500 [==============================] - 21s 9ms/sample - loss: 10.2951 - accuracy: 0.0000e+00 - val_loss: 9.0881 - val_accuracy: 0.0000e+00
Epoch 2/20
2500/2500 [==============================] - 19s 8ms/sample - loss: 7.6635 - accuracy: 0.0000e+00 - val_loss: 9.5025 - val_accuracy: 0.0000e+00
Epoch 3/20
2500/2500 [==============================] - 14s 6ms/sample - loss: 5.5616 - accuracy: 0.0000e+00


KeyboardInterrupt: 

In [123]:
submission=model.predict(X)



KeyboardInterrupt: 

In [110]:
df = pd.DataFrame()
df["market_change%"]=submission
df["News_ID"]=data["News_ID"]

In [108]:
s=submission.shape[0]

In [109]:
submission =submission.reshape(s)

In [125]:
s=model.evaluate(X[4000:],y[4000:])

2000/2000 [==============================] - 7s 4ms/sample - loss: 1.7501 - accuracy: 0.0000e+00


In [127]:
s=model.predict(X[4500:])

In [129]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y[4500:],s)

6.44140047067247

In [ ]:
from keras.preprocessing.text import Tokenizer

In [111]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['joined'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 55644 unique tokens.


In [112]:
X = tokenizer.texts_to_sequences(data['joined'].values)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (6000, 250)


In [113]:
Y=label.values

In [114]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import  Embedding, LSTM

In [115]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
# model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='adam',metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X, Y, epochs=epochs, batch_size=batch_size,validation_split=0.1)


Train on 5400 samples, validate on 600 samples
Epoch 1/5
5400/5400 [==============================] - 137s 25ms/sample - loss: 2.3428 - accuracy: 0.0000e+00 - val_loss: 2.2895 - val_accuracy: 0.0000e+00
Epoch 2/5
5400/5400 [==============================] - 182s 34ms/sample - loss: 2.3100 - accuracy: 0.0000e+00 - val_loss: 2.3101 - val_accuracy: 0.0000e+00
Epoch 3/5
5400/5400 [==============================] - 184s 34ms/sample - loss: 2.1876 - accuracy: 0.0000e+00 - val_loss: 2.4395 - val_accuracy: 0.0000e+00
Epoch 4/5
5400/5400 [==============================] - 196s 36ms/sample - loss: 1.9150 - accuracy: 0.0000e+00 - val_loss: 2.5711 - val_accuracy: 0.0000e+00
Epoch 5/5
5400/5400 [==============================] - 194s 36ms/sample - loss: 1.6506 - accuracy: 0.0000e+00 - val_loss: 2.5916 - val_accuracy: 0.0000e+00


In [116]:
model.predict(X[500:502])

array([[-1.9335413 ],
       [ 0.10406708]], dtype=float32)

In [117]:
Y[500:502]

array([-0.49005691,  0.37257164])

In [118]:
a = [1, 2, 3, 4, 5]

In [119]:
b = [6, 7, 8, 9]

In [120]:
a.append(b)

In [121]:
a.extend(b)

In [122]:
a

[1, 2, 3, 4, 5, [6, 7, 8, 9], 6, 7, 8, 9]